In [161]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import register

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from collections import namedtuple

In [162]:
class TradeEnv(gym.Env):
    
    def __init__(self, input_dim, max_steps, cash_balance):
        super(TradeEnv, self).__init__()
        self.input_dim = input_dim
        self.action_space = spaces.Discrete(3)  # actions: buy, sell and hold
        self.observation_space = spaces.Box(low=np.zeros(self.input_dim), high=np.ones(self.input_dim) * np.inf, dtype=np.float64)

        self.current_step = 0
        self.max_steps = max_steps  # episode terminates when day ends

        self.state = None

        self.hold_counter = 0
        self.impossible_sell_counter = 0
        self.current_price = 0
        self.initial_cash = cash_balance

        self.inventory = {'Last Buy Price': 0,
                          'Holdings': 0,
                          'Cash Balance': cash_balance,
                          'Profit': 0,
                          'Volatility': 0,
                          'Sharpe Ratio': 0}
        

    def step(self, action):
        self.current_step += 1
        done = self.current_step == self.max_steps

        reward = self.calculate_pnl(action)
        self.state = np.zeros(self.input_dim, np.float32)

        info = {"Profit": self.inventory["Profit"]}

        return self.state, reward, done, info
    
    def update_state(self, next_state):
        self.state = next_state
        return self.state
    

    def reset(self, options):
        self.__init__(self.input_dim, self.max_steps, self.initial_cash)
        self.state = options["initial state"]

        info = {}

        return self.state, info


    def calculate_pnl(self, action):

        self.current_price = self.state[0]  # select open price for OHLCVT data

        if action == 0: # buy
            buy_quantity = self.inventory["Cash Balance"] / self.current_price
            self.inventory['Cash Balance'] = 0
            self.inventory["Holdings"] = buy_quantity
            self.hold_counter = 0
            self.inventory["Last Buy Price"] = self.current_price
            reward = 0
            self.buy_flag = True

        elif action == 1: # sell
            if self.inventory["Holdings"] > 0:
                self.inventory["Cash Balance"] += self.inventory["Holdings"] * self.current_price
                reward = self.inventory["Cash Balance"] - self.inventory["Holdings"] * self.inventory["Last Buy Price"]
                self.inventory["Holdings"] = 0
                self.inventory["Last Buy Price"] = 0
                self.hold_counter = 0
                self.inventory['Profit'] += reward
            else:
                self.impossible_sell_counter += 1
                reward = -self.impossible_sell_counter

        elif action == 2: # hold
            self.hold_counter += 1
            reward = -self.hold_counter / self.initial_cash

        return reward



In [147]:
register(
     id="TradeEnv-v1",
     entry_point="__main__:TradeEnv",
     kwargs={'input_dim': 6, 'max_steps': 288, 'cash_balance': 1000}
)

/Users/wafic/miniconda3/envs/datalibs/lib/python3.11/site-packages/gymnasium/envs/registration.py:498: UserWarning: WARN: Overriding environment TradeEnv-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [148]:
env = gym.make('TradeEnv-v1')

# Test

In [152]:
df = pd.read_csv('ETHUSD_5.csv', header=None, names=['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Trades'])
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df.set_index('Time', inplace=True)
df = df.to_numpy()


In [153]:
init_state = df[0]
init_state

array([1934.02      , 1936.57      , 1933.66      , 1935.45      ,
        121.35739837,   40.        ])

In [154]:
first_state, _ = env.reset(options={"initial state": init_state})

In [155]:
next_state, reward, done, info  = env.step(0)

/Users/wafic/miniconda3/envs/datalibs/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float64, actual type: float32
  logger.warn(


In [156]:
next_state = env.update_state(df[1])

In [157]:
next_state

array([1935.91      , 1937.2       , 1934.58      , 1936.95      ,
          2.38832811,   14.        ])

In [158]:
next_state, reward, done, info  = env.step(1)

In [159]:
env.inventory

{'Last Buy Price': 0,
 'Holdings': 0,
 'Cash Balance': 1000.97723911852,
 'Profit': 0.9772391185200604,
 'Volatility': 0,
 'Sharpe Ratio': 0}

In [160]:
reward

0.9772391185200604